In [1]:
!pip install pyngrok

In [10]:
import json
from flask import Flask, render_template, request
from transformers import pipeline
from pyngrok import ngrok

In [3]:
!ngrok config add-authtoken 2gtJMBzQXOOGE9Eu3VXkvcdZfue_3i26umVYWCEumkYyxkqfE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [4]:
qa_model = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/451 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
def load_rules():
    with open("business_rules.json", "r") as file:
        data = json.load(file)
    return data["rules"]

In [13]:
global context

context = load_rules()

app = Flask(__name__)

In [7]:
@app.route("/", methods=["GET", "POST"])
def home():
    global context
    if request.method == "POST":
        question = request.form["question"]

        result = qa_model(question=question, context=context)
        answer = result["answer"]

        return render_template("index.html", question=question, answer=answer)

    return render_template("index.html", answer="")

In [14]:
@app.route("/update_context", methods=["POST"])
def update_context():
    new_rule = request.form["new_rule"]
    context += f" {new_rule}"

    with open("business_rules.json", "w") as file:
        json.dump({"rules": context}, file)

    return "Contexto atualizado com sucesso!", 200

In [15]:
public_url = ngrok.connect(5000)
print(f"API está disponível em: {public_url}")

# Iniciando o servidor Flask
app.run(host="0.0.0.0", port=5000)


API está disponível em: NgrokTunnel: "https://ab59-104-198-15-135.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
